In [1]:
#导入数据
import pandas as pd
import pymysql
import numpy as np

def conn_sql():
    conn = pymysql.connect(
                        host='localhost',
                        port=3306,
                        user='root',
                        passwd='941207',
                        db='yeyw_db',
                        use_unicode=True,
                        charset='utf8',
                        )
    sql_cmd = "SELECT * FROM commission_channel_info_08"
    data = pd.read_sql(sql_cmd, conn)
    return data
    
data = conn_sql()

In [2]:
#处理缺失值
data.loc[data.org_channel_type == '直销渠道', 'star_level'] = '3'
data.loc[data.star_level == '', 'star_level'] = np.NaN

data['star_level'] = data['star_level'].fillna(method='ffill')
data['star_level'] = data['star_level'].astype('int')


In [3]:
data = data[['object_id', 'sum_money_ratio', 'arpu_ratio', 'star_level', 
             'innet_user', 'keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 'month_off_user','old_user_num',
             'batch_open_num', 'err_time_busi', 'open_acct_num','org_user_numm', 'dkm_hb', 'dkm_busi_ratio', 'same_money',
             'same_money_ratio', 'recount_flag', 'recount_user', 'recount_fee', 'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc', 
             'zd_yw_lc_ratio']]




print(data.head(10))

  object_id  sum_money_ratio  arpu_ratio  star_level  innet_user  \
0  11000553           1.0000     -0.0370           1      0.9000   
1  11001989          -0.4108     -0.1057           2      0.9214   
2  11007992          -0.4116     -0.0901           2      0.8885   
3  11008564           0.5474     -0.0940           3      0.8358   
4  11008616           0.0287     -0.0657           5      0.8516   
5  11008799           0.1959     -0.1571           2      0.8768   
6  11009084          -0.2915     -0.1069           2      0.8721   
7  11009136          -0.6593     -0.1420           1      0.8684   
8  11010161          -0.6549     -0.2453           1      0.9421   
9  11010213          -0.2221     -0.0213           3      0.7288   

   keep_card_user  dmrn_mark  nonefee_user  lowfee_user  month_off_user  \
0          0.0000     0.0148        0.1778       0.0296          0.0259   
1          0.0000     0.0111        0.2104       0.0111          0.0116   
2          0.0006     0.01

In [4]:
#对部分字段做log变换
indexList=['keep_card_user', 'dmrn_mark', 'nonefee_user','lowfee_user', 'month_off_user','old_user_num',
           'batch_open_num', 'err_time_busi', 'open_acct_num','org_user_numm','dkm_busi_ratio', 'same_money_ratio',
           'recount_user', 'recount_fee', 'zd_yw_jc_ratio', 'zd_yw_lc_ratio']


# indexList=[ 'keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 'month_off_user','old_user_num',
#              'batch_open_num', 'err_time_busi', 'open_acct_num','org_user_numm', 'dkm_hb', 'dkm_busi_ratio', 'same_money',
#              'same_money_ratio', 'recount_flag', 'recount_user', 'recount_fee', 'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc', 
#              'zd_yw_lc_ratio']


indexList2=['sum_money_ratio', 'arpu_ratio'] #暂时没有log也没有异常检测

for index in indexList:
    print('======================'+index+'==========================') 
    print(data[index].skew())
    data[index] = np.log(data[index] + 1e-5) 
    print(data[index].skew())
    
print(data.head(10))
# data.to_csv("data.csv", header=True, encoding="gbk")

======================keep_card_user==========================
15.124614330304357
0.6701775891832311
======================dmrn_mark==========================
5.19310792516471
-0.5261102821786319
======================nonefee_user==========================
1.511801018906337
-3.2233396671217265
======================lowfee_user==========================
19.444393666652516
-1.7565018231220795
======================month_off_user==========================
13.288409016447376
-1.0138935112588439
======================old_user_num==========================
32.547361936514875
0.4315245273905428
======================batch_open_num==========================
31.626270322306688
14.224873571989733
======================err_time_busi==========================
94.98400588430952
7.710584503616911
======================open_acct_num==========================
23.472732897237318
1.047224903440372
======================org_user_numm==========================
8.605375270686318
-0.9788170972222688
=======

In [57]:
#判断每个字段上下界限

'''for item in indexList:
    print('=================='+item+'=========================')
    df = data[item][round(data[item], 4) != -11.5129]
    iqr = df.quantile(0.75) - df.quantile(0.25)
    q_abnormal_L = df < df.quantile(0.25) - 1.5 * iqr
    q_abnormal_U = df > df.quantile(0.75) + 1.5 * iqr
    print(item + '中有' + str(q_abnormal_L.sum() + q_abnormal_U.sum()) + '个异常值')
#     print('上界='+str(df.quantile(0.75) + 1.5 * iqr)+'--------下界限='+str(df.quantile(0.25) - 1.5 * iqr))
'''

"for item in indexList:\n    print('=================='+item+'=========================')\n    df = data[item][round(data[item], 4) != -11.5129]\n    iqr = df.quantile(0.75) - df.quantile(0.25)\n    q_abnormal_L = df < df.quantile(0.25) - 1.5 * iqr\n    q_abnormal_U = df > df.quantile(0.75) + 1.5 * iqr\n    print(item + '中有' + str(q_abnormal_L.sum() + q_abnormal_U.sum()) + '个异常值')\n#     print('上界='+str(df.quantile(0.75) + 1.5 * iqr)+'--------下界限='+str(df.quantile(0.25) - 1.5 * iqr))\n"

In [29]:
#处理异常点  作为缺失值处理
'''
# narList=['keep_card_user', 'err_time_busi', 'open_acct_num', 'same_money_ratio', 'recount_user','recount_fee','zd_yw_jc_ratio']
# widList=['dmrn_mark', 'lowfee_user', 'month_off_user','old_user_num','org_user_numm']


indexList=['keep_card_user', 'dmrn_mark', 'nonefee_user','lowfee_user', 'month_off_user','old_user_num',
           'batch_open_num', 'err_time_busi', 'open_acct_num','org_user_numm','dkm_busi_ratio', 'same_money_ratio',
           'recount_user', 'recount_fee', 'zd_yw_jc_ratio', 'zd_yw_lc_ratio']

# indexList=[ 'keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 'month_off_user','old_user_num',
#              'batch_open_num', 'err_time_busi', 'open_acct_num','org_user_numm', 'dkm_hb', 'dkm_busi_ratio', 'same_money',
#              'same_money_ratio', 'recount_flag', 'recount_user', 'recount_fee', 'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc', 
#              'zd_yw_lc_ratio']

indexList2=['sum_money_ratio', 'arpu_ratio'] #暂时没有log也没有异常检测




for item in indexList:
    print('=================='+item+'=========================')
    df = data[item][round(data[item], 4) != -11.5129]
    iqr = df.quantile(0.75) - df.quantile(0.25)
    downLine = df.quantile(0.25) - 1.5 * iqr
    upLine = df.quantile(0.75) + 1.5 * iqr
    
    data.loc[(data[item] > upLine) & (round(data[item], 4) != -11.5129), item] = upLine
    data.loc[(data[item] < downLine) & (round(data[item], 4) != -11.5129), item] = downLine


for item in indexList2:
    print('=================='+item+'=========================')
    df = data[item][round(data[item], 4) != -11.5129]
    iqr = df.quantile(0.75) - df.quantile(0.25)
    
    downLine = df.quantile(0.25) - 2 * iqr
    upLine = df.quantile(0.75) + 2 * iqr
    
    data.loc[(data[item] > upLine) & (round(data[item], 4) != -11.5129), item] = upLine
    data.loc[(data[item] < downLine) & (round(data[item], 4) != -11.5129), item] = downLine
    
# data.to_csv("data3.csv", header=True, encoding="gbk")

'''

'\n# narList=[\'keep_card_user\', \'err_time_busi\', \'open_acct_num\', \'same_money_ratio\', \'recount_user\',\'recount_fee\',\'zd_yw_jc_ratio\']\n# widList=[\'dmrn_mark\', \'lowfee_user\', \'month_off_user\',\'old_user_num\',\'org_user_numm\']\n\n\nindexList=[\'keep_card_user\', \'dmrn_mark\', \'nonefee_user\',\'lowfee_user\', \'month_off_user\',\'old_user_num\',\n           \'batch_open_num\', \'err_time_busi\', \'open_acct_num\',\'org_user_numm\',\'dkm_busi_ratio\', \'same_money_ratio\',\n           \'recount_user\', \'recount_fee\', \'zd_yw_jc_ratio\', \'zd_yw_lc_ratio\']\n\n# indexList=[ \'keep_card_user\', \'dmrn_mark\', \'nonefee_user\', \'lowfee_user\', \'month_off_user\',\'old_user_num\',\n#              \'batch_open_num\', \'err_time_busi\', \'open_acct_num\',\'org_user_numm\', \'dkm_hb\', \'dkm_busi_ratio\', \'same_money\',\n#              \'same_money_ratio\', \'recount_flag\', \'recount_user\', \'recount_fee\', \'zd_yw_jc\', \'zd_yw_jc_ratio\', \'zd_yw_lc\', \n#          

In [5]:
#标准化处理
data.set_index('object_id', inplace=True)

dataNeg = data[['star_level', 'innet_user', 'org_user_numm']]

# dataPos = data[['sum_money_ratio', 'arpu_ratio','keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 
#                 'month_off_user','old_user_num', 'batch_open_num', 'err_time_busi', 'open_acct_num', 'dkm_hb',
#                 'dkm_busi_ratio', 'same_money','same_money_ratio', 'recount_flag', 'recount_user', 'recount_fee', 
#                 'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc',  'zd_yw_lc_ratio']]

dataPos = data[['sum_money_ratio','keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 
                'month_off_user','old_user_num', 'batch_open_num', 'err_time_busi', 'open_acct_num',]]

dataPos2 = data[['arpu_ratio', 'dkm_hb', 'dkm_busi_ratio', 'same_money','same_money_ratio', 'recount_flag', 'recount_user', 'recount_fee', 
                'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc',  'zd_yw_lc_ratio']]



# 正向指标
dataPos = (dataPos - dataPos.min())/(dataPos.max() - dataPos.min())

dataPos2 = (dataPos2 - dataPos2.min())/(dataPos2.max() - dataPos2.min())

# dataPos2 = dataPos2 * 9 + 1

# 负向指标
dataNeg = (dataNeg.max() - dataNeg)/(dataNeg.max() - dataNeg.min())
    
    

df = pd.merge(dataPos, dataNeg, how='inner', left_index = True, right_index=True)
            
df = pd.merge(df, dataPos2, how='inner', left_index = True, right_index=True)

df = df.fillna(0)

df = df * 9 + 1

# df = df[['star_level', 'innet_user', 'org_user_numm', 'keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 
#          'month_off_user', 'sum_money_ratio', 'old_user_num', 'batch_open_num', 'err_time_busi', 'open_acct_num', 
#          'arpu_ratio', 'dkm_hb','dkm_busi_ratio', 'same_money','same_money_ratio', 'recount_flag', 'recount_user', 
#          'recount_fee', 'zd_yw_jc', 'zd_yw_jc_ratio', 'zd_yw_lc',  'zd_yw_lc_ratio']]

df = df[['star_level', 'innet_user', 'org_user_numm', 'keep_card_user', 'dmrn_mark', 'nonefee_user', 'lowfee_user', 'month_off_user', 'sum_money_ratio', 'old_user_num', 'batch_open_num', 'err_time_busi', 'open_acct_num', 'arpu_ratio', 'dkm_busi_ratio', 'same_money_ratio', 'recount_user', 'recount_fee', 'zd_yw_jc_ratio', 'zd_yw_lc_ratio']]


df.to_csv("dfStandard.csv", header=True, encoding="gbk")

In [6]:
# 求k
import math

df2 = df
rows = df2.index.size  # 行
cols = df2.columns.size  # 列
k = 1.0 / math.log(rows)
    
print(rows)
print(cols)
print(k)

10868
20
0.10760118472877289


In [82]:
 # 矩阵计算--
# 信息熵
from numpy import *


df4 = array(df2)
lnf = [[None] * cols for i in range(rows)]
lnf = array(lnf)
for i in range(rows):
    for j in range(cols):
        if df4[i][j] == 0:
            lnfij = 0.0
        else:
            p = df4[i][j] / df4.sum(axis=0)[j]
            lnfij = math.log(p) * p * (-k)
        lnf[i][j] = lnf
lnf = pd.DataFrame(lnf)
# lnf.to_csv("Result.csv", header=True, encoding="gbk")   


# 计算冗余度
E = lnf
d = 1 - E.sum(axis=0)
res = pd.DataFrame(d)
print(type(d))
lnf.to_csv("lnf.csv", header=True, encoding="gbk")
res.to_csv("Result.csv", header=True, encoding="gbk")

KeyboardInterrupt: 

In [9]:
#用的这个

def Entropy (data):
    df3 = pd.DataFrame(data = None, columns = data.columns, index = data.index)
    df4 = pd.DataFrame(data = None, columns = data.columns, index = data.index)

    for index, row in data.iteritems():
        indexSum = data[index].sum()
        for i in range(0,len(row)): 
            if data[index][i] == 0:
                lnfij2 = 0.0
            else:           
                p2 = data[index][i] / indexSum
                #注：第一次熵值法0。00001可以不加，第二次必须要加；第一次到底加不加看后期模型效果
                lnfij2 = math.log(p2+ 0.00001) * p2 * (-k)
            df4[index][i] = p2
            df3[index][i] = lnfij2   
        
    # 计算冗余度
    E = df3
    d = 1 - E.sum(axis=0)
    return d, df4
    
d, p =  Entropy(df2)
# p.to_csv("p2.csv", header=True, encoding="gbk")        
# d.to_csv("Result2.csv", header=True, encoding="gbk")        

df3的内存地址：2596589110608
df4的内存地址：2596589110608
data的内存地址：2596589110608


KeyboardInterrupt: 

In [18]:
#计算第二层权重
d.columns = ['value']

channel = d.star_level + d.innet_user + d.org_user_numm
improve = d.keep_card_user + d.dmrn_mark + d.nonefee_user + d.lowfee_user + d.month_off_user
busi = d.sum_money_ratio + d.old_user_num + d.batch_open_num + d.err_time_busi + d.open_acct_num
money = d.arpu_ratio + d.dkm_busi_ratio + d.same_money_ratio + d.recount_user + d.recount_fee + d.zd_yw_jc_ratio + d.zd_yw_lc_ratio

d.star_level = d.star_level / channel
d.innet_user = d.innet_user / channel
d.org_user_numm = d.org_user_numm / channel

d.keep_card_user = d.keep_card_user / improve
d.dmrn_mark = d.dmrn_mark / improve
d.nonefee_user = d.nonefee_user / improve
d.lowfee_user = d.lowfee_user / improve
d.month_off_user = d.month_off_user / improve

d.sum_money_ratio = d.sum_money_ratio / busi
d.old_user_num = d.old_user_num / busi
d.batch_open_num = d.batch_open_num / busi
d.err_time_busi = d.err_time_busi / busi
d.open_acct_num = d.open_acct_num / busi

d.arpu_ratio = d.arpu_ratio / money
d.dkm_busi_ratio = d.dkm_busi_ratio / money
d.same_money_ratio = d.same_money_ratio / money
d.recount_user = d.recount_user / money
d.recount_fee = d.recount_fee / money
d.zd_yw_jc_ratio = d.zd_yw_jc_ratio / money
d.zd_yw_lc_ratio = d.zd_yw_lc_ratio / money

print(d)

star_level          0.317710
innet_user          0.406746
org_user_numm       0.275544
keep_card_user      0.324726
dmrn_mark           0.240990
nonefee_user        0.106663
lowfee_user         0.144805
month_off_user      0.182816
sum_money_ratio     0.082271
old_user_num        0.308543
batch_open_num      0.116453
err_time_busi       0.138872
open_acct_num       0.353862
arpu_ratio          0.060429
dkm_busi_ratio      0.099184
same_money_ratio    0.116771
recount_user        0.236690
recount_fee         0.353776
zd_yw_jc_ratio      0.063713
zd_yw_lc_ratio      0.069437
dtype: float64


In [19]:
#第二次熵值法
dfSec = pd.DataFrame(columns = ['channel', 'improve', 'busi', 'money'])
dfSec['channel'] = p.star_level * d.star_level + p.innet_user * d.innet_user + p.org_user_numm * d.org_user_numm
dfSec['improve'] = p.keep_card_user * d.keep_card_user + p.dmrn_mark * d.dmrn_mark + p.nonefee_user * d.nonefee_user + p.lowfee_user * d.lowfee_user + p.month_off_user * d.month_off_user
dfSec['busi'] = p.sum_money_ratio * d.sum_money_ratio + p.old_user_num * d.old_user_num + p.batch_open_num * d.batch_open_num + p.err_time_busi * d.err_time_busi + p.open_acct_num * d.open_acct_num
dfSec['money'] = p.arpu_ratio * d.arpu_ratio + p.dkm_busi_ratio * d.dkm_busi_ratio + p.same_money_ratio * d.same_money_ratio + p.recount_user * d.recount_user + p.recount_fee * d.recount_fee + p.zd_yw_jc_ratio * d.zd_yw_jc_ratio + p.zd_yw_lc_ratio * d.zd_yw_lc_ratio

# dfSec.to_csv("dfSec.csv", header=True, encoding="gbk")


dRes, pRes = Entropy(dfSec)
# pRes.to_csv("pRes.csv", header=True, encoding="gbk")        
# dRes.to_csv("dRes.csv", header=True, encoding="gbk")
    



In [20]:

dRes  = dRes/dRes.sum()
print('权重为：')
print(dRes)
dfScore = pd.DataFrame(columns = ['score'])
dfScore['score'] = dRes.channel * pRes.channel + dRes.improve * pRes.improve + dRes.busi * pRes.busi + dRes.money * pRes.money


dfScore = (dfScore - dfScore.min())/(dfScore.max() - dfScore.min()) * 100

dfScore.to_csv("dfScore.csv", header=True, encoding="gbk") 

权重为：
channel    0.169113
improve    0.239378
busi       0.263095
money      0.328414
dtype: float64
